In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer

# Load tokenizer and model
model_name = "woctordho/lojban-translation"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Enable GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"Using device: {device}")

def translate_to_lojban(text, max_length=100):
    """
    Translate English text to Lojban using MarianMT
    Args:
        text (str): Input English text
        max_length (int): Maximum length of generated translation
    Returns:
        str: Translated Lojban text
    """
    # Prepare the input and move to the same device as the model
    encoded = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    encoded = {k: v.to(device) for k, v in encoded.items()}  # Move all tensors to the correct device
    
    # Generate translation
    with torch.no_grad():
        outputs = model.generate(
            **encoded,
            max_length=max_length,
            num_beams=5,
            length_penalty=1.0,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Move outputs back to CPU for decoding if necessary
    outputs = outputs.cpu()
    
    # Decode the generated text
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Test the translation
test_text = "Hello, how are you?"
print(f"English: {test_text}")
translation = translate_to_lojban(test_text)
print(f"Lojban: {translation}")